```{r setup, include=FALSE}
knitr::opts_chunk$set(echo = TRUE)
reticulate::use_python("/usr/local/bin/python3")
```

**Data cleaning** is the process of fixing or removing incorrect, corrupted, incorrectly formatted, duplicate, or incomplete data within a dataset. When combining multiple data sources, there are many opportunities for data to be duplicated or mislabeled. If data is incorrect, outcomes and algorithms are unreliable, even though they may look correct. There is no one absolute way to prescribe the exact steps in the data cleaning process because the processes will vary from dataset to dataset. But it is crucial to establish a template for your data cleaning process so you know you are doing it the right way every time. In this section, I will clean my text data and modified data. The steps for cleaning these two kind of data is different, so I will specify the cleaning steps with presenting the code. 

## Part 1 - R on downloaded dataset

Load the possible packages
```{r}
library(tidyverse)
library(dplyr)
library(reshape2)
library(tidyr)
library(stringr)
```

#### Energy Consumption by sector dataset

This dataset contains the info of value of monthly energy consumption by different sectors. I will use this official data set from U.S. EIA to see how exact the monthly usage of different energy in different sectors. Since the dataset is too dirty and I cannot tell what kind of variables it has now, let's do the cleaning and tidying step by step. Finally, I will introduce the variables I have in the dataset.

 - Step 1: load the dataset
```{r}
sector <- read.csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/downloaded data/consumptionbysector.csv")
head(sector,5)
```

 - Step 2: check the NA missing value. If have, remove it. 

 Through checking, No NA missing value in the dataset
```{r}
sum(is.na(sector))
```

 - Step 3: Remove the irrelevent columns
 ```{r}
sector.clean <- sector[ , ! names(sector) %in% c("MSN","Column_Order")]
head(sector.clean)
 ```
 
 - Step 4: drop the value in column

 Since the column "Description" in this dataset contain the info of energy type and sector type AND there are more than 10000 dataset, I want to remove the irrelevent row with some value in this column.

 I check the levels of the column and there are totally 15 levels. Among the levels, there are lots of contains that I do not need. 
 ```{r}
 unique(sector$Description)
 ```

 Remove the value i do not need in "Description"
 ```{r}
sector.clean <- sector.clean %>%
         filter( Description != "Electricity Sales to Ultimate Customers in the Residential Sector")%>%
         filter( Description != "Electricity Sales to Ultimate Customers in the Commercial Sector")%>%
         filter( Description != "Electricity Sales to Ultimate Customers in the Industrial Sector")
 ```

 - Step 5: split the column "Description" into two columns "sector" and "energy type"
 ```{r}
 sector.clean[c('energy_type', 'sector')] <- str_split_fixed(sector.clean$Description, 'by', 2)
 ```

 - Step 6: change the variable levels name and drop the column "Description"
 ```{r}
 sector.clean$energy_type[sector.clean$energy_type=="Primary Energy Consumed"]<-"Primary Energy"
 sector.clean$energy_type[sector.clean$energy_type=="End-Use Energy Consumed"]<-"End-Use Energy"
 sector.clean$energy_type[sector.clean$energy_type=="Electrical System Energy Losses"]<-"Electrical Energy"
 sector.clean$sector[sector.clean$sector=="the Residential Sector"]<-"Residential"
 sector.clean$sector[sector.clean$sector=="the Commercial Sector"]<-"Commercial"
 sector.clean$sector[sector.clean$sector=="the Industrial Sector"]<-"Industrial"

 sector.clean <- sector.clean[ , ! names(sector.clean) %in% c("Description")]
 head(sector.clean)
 ```
 - Step 7: save to csv file
 ```{r}
 write.csv(sector.clean,"/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/cleaned data/cleansector.csv")
 ```

In [1]:
import pandas as pd
cleansector = pd.read_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/cleaned data/cleansector.csv",index_col=[0])
cleansector.head()

,YYYYMM,Value,Unit,energy_type,sector
1,197301,1339.270,Trillion Btu,Primary Energy Consumed,the Residential Sector
2,197302,1174.899,Trillion Btu,Primary Energy Consumed,the Residential Sector
3,197303,983.374,Trillion Btu,Primary Energy Consumed,the Residential Sector
4,197304,715.391,Trillion Btu,Primary Energy Consumed,the Residential Sector
5,197305,535.914,Trillion Btu,Primary Energy Consumed,the Residential Sector


## Part 2 - Python on Text Data 

#### Dataset "twitterpython.csv" which is grabbed through python

People's perception of energy consumption is very important. It not only reflects the public's perception, but also indirectly reflects people's concern for environmental issues and their willingness to change the status quo. In the data gathering section, I have gather the text data by setting “energy consumption” “global warming” “clean energy” “conventional energy” “environmental protection” as the keywords for the twitter api to get the text data. Now I will do the data processing for the text dataset through python. 

 - Step 1: import the packages we may use during data processing

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import svm
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment import SentimentIntensityAnalyzer

 - Step 2: Load the Dataset we grab by using python

In [4]:
twitter = pd.read_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/raw data/twitterpython.csv",index_col=[0])
twitter.head()

,author_id,text,created_at,id,edit_history_tweet_ids,lang
0,1549857918442524672,@bjgalloway1717 @abbieasr Global warming? Thou...,2022-12-07T10:25:57.000Z,1600436443490328576,['1600436443490328576'],en
1,1075616751663104000,@shirtsthtgohard i'm pretty excited for global...,2022-12-07T10:25:51.000Z,1600436419838717952,['1600436419838717952'],en
2,1515416161281785859,@JustinTrudeau What kind of idiot would burden...,2022-12-07T10:25:42.000Z,1600436381263663104,['1600436381263663104'],en
3,105241093,What beautiful but 'scary' visualisation of ea...,2022-12-07T10:25:31.000Z,1600436336145465344,['1600436336145465344'],en
4,18852344,@DawnNeesom Don't forget if it Snows in Decemb...,2022-12-07T10:25:24.000Z,1600436304793153536,['1600436304793153536'],en


 - Step 3: Find the dataset with NA missing value
 
 Through checking, there is no missing value in the text dataset.

In [5]:
twitter.isnull().sum()

author_id                 0
text                      0
created_at                0
id                        0
edit_history_tweet_ids    0
lang                      0
dtype: int64

 - Step 4: Remove data with language other than English

 Since my Twitter API collect the text with different languages, I need to drop the text information that cannot be understand by me and you. I only keep the text with English.

In [6]:
twitter = twitter[twitter.lang == "en"]
twitter

,author_id,text,created_at,id,edit_history_tweet_ids,lang
0,1549857918442524672,@bjgalloway1717 @abbieasr Global warming? Thou...,2022-12-07T10:25:57.000Z,1600436443490328576,['1600436443490328576'],en
1,1075616751663104000,@shirtsthtgohard i'm pretty excited for global...,2022-12-07T10:25:51.000Z,1600436419838717952,['1600436419838717952'],en
2,1515416161281785859,@JustinTrudeau What kind of idiot would burden...,2022-12-07T10:25:42.000Z,1600436381263663104,['1600436381263663104'],en
3,105241093,What beautiful but 'scary' visualisation of ea...,2022-12-07T10:25:31.000Z,1600436336145465344,['1600436336145465344'],en
4,18852344,@DawnNeesom Don't forget if it Snows in Decemb...,2022-12-07T10:25:24.000Z,1600436304793153536,['1600436304793153536'],en
...,...,...,...,...,...,...
95,152971284,You lot. You spend all your time thinking abou...,2022-12-07T10:02:28.000Z,1600430533154025472,['1600430533154025472'],en
96,3402582670,"RT @Dad_At_Law: First voicemail, next up, glob...",2022-12-07T10:02:27.000Z,1600430530016657408,['1600430530016657408'],en
97,1181913812657565707,for the first time ever; i’m finally getting w...,2022-12-07T10:02:26.000Z,1600430526690562048,['1600430526690562048'],en
98,947078072595034112,"@Lindhacker @ZLabe @NOAA_ESRL Re: ""Thermal dyn...",2022-12-07T10:01:17.000Z,1600430235165810689,['1600430235165810689'],en


 - Step 4: Pre-processing the text info in the dataset
 
 Since the raw data contains lots of puntuations, urls, commas, numbers, highercase, and some other things which will influence our analysis and tokenization. Therefore, we need to remove the unnecessary things. Since this semester, I take Natural Language Processing course, and learn how to use **pipline** to process the text information. So, in this part, I will the method I learn from NLP. 
 
 Through pipline, the things which will influence our analysis will be cleaned and replaced. And then, we will need to transform the sentence from text data to tokens which is a list of words through tokenizer.

In [7]:
import re
import spacy
from spacy.language import Language

pipeline = spacy.load('en_core_web_sm')

# http://emailregex.com/
email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

# replace = [ (pattern-to-replace, replacement),  ...]
replace = [
    (r"<a[^>]*>(.*?)</a>", r"\1"),  # Matches most URLs
    (email_re, "email"),            # Matches emails
    (r"(?<=\d),(?=\d)", ""),        # Remove commas in numbers
    (r"\d+", "number"),              # Map digits to special token <numbr>
    (r"[\t\n\r\*\.\@\,\-\/]", " "), # Punctuation and other junk
    (r"\s+", " ")                   # Stips extra whitespace
]

twitter_sentences = []
for i, d in enumerate(twitter['text']):
    for repl in replace:
        d = re.sub(repl[0], repl[1], d)
    twitter_sentences.append(d)

@Language.component("lab04Preprocessor")
def ng20_preprocess(doc):
    tokens = [token for token in doc 
              if not any((token.is_stop, token.is_punct))]
    tokens = [token.lemma_.lower().strip() for token in tokens]
    tokens = [token for token in tokens if token]
    return " ".join(tokens)


pipeline.add_pipe("lab04Preprocessor")

<function __main__.ng20_preprocess(doc)>

 - Step 5: Pass data through spacy pipeline

In [1]:
docs = []
for sent in twitter_sentences:
    docs.append(pipeline(sent))

result = pd.DataFrame(docs)
result.head()

NameError: name 'twitter_sentences' is not defined

 - Step 6: Backup the processed data

In [ ]:
result.to_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/cleaned data/piplineresult.csv")


 - Step 7: Using the CountVectorizer and count the words freqency
 
 Countvectorizer can convert a collection of text documents to a matrix of token counts.

In [ ]:
vectorizer = CountVectorizer()
bow = vectorizer.fit(docs)
features = bow.vocabulary_.keys()
counts = bow.vocabulary_.values()
bow=pd.DataFrame({'words':features,'counts':counts})
bow = bow.sort_values(by=['counts'],ascending=False)
bow

 - Step 8: Backup the processed data 

In [ ]:
bow.to_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/cleaned data/countword.csv")

 - Step 9: Opinion Mining
 
 Opinion mining is an approach to natural language processing (NLP) that identifies the emotional tone behind a body of text. This is a popular way for organizations to determine and categorize opinions about a product, service, or idea. In addition to identifying sentiment, opinion mining can extract the polarity (or the amount of positivity and negativity), subject and opinion holder within the text. Furthermore, sentiment analysis can be applied to varying scopes such as document, paragraph, sentence and sub-sentence levels.

 Since I would like to analysis the attitude people toward the energy consumption, opinion mining is an useful methods to conduct the result. 

In [ ]:
# define the function
defgetSentiments(df):
   sid = SentimentIntensityAnalyzer()
   tweet_str = ""
   tweetscore = []
   for tweet in df['text']:
       tweet_str = tweet_str + " " + tweet
       score = sid.polarity_scores(tweet_str)
       tweetscore.append(score)
   return tweetscore

#call the function above to see the result score and form a data frame to record
sentiment= getSentiments(twitter)
texts= pd.DataFrame(twitter.text)
stmscore= pd.DataFrame.from_dict(sentiment)
stmscore.head()

#relate the text and score for better view
txtscore= pd.concat([texts,stmscore],axis=1)
txtscore.head()

#export the csv for future analysis
txtscore.to_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/cleaneddata/pystmscore.csv")

#### Dataset "twitterR.csv" which is grabbed by R

For Twitter API in R, I used twitter to scratch keywords like **"oil","gas","solar power","wind power"** to scratch users’ attitudes about these four kinds of energy. I want to search different types of energy in order to analysis people attitude toward different enegy, and what they think about two different kind of energy (renewable energy and convential energy).

 - Step 1: import the packages we may use during data processing
 ```{r}
 library(selectr)
library(rvest)
library(xml2)
library(wordcloud2) # for generating really cool looking wordclouds
library(tm) # for text minning
library(dplyr) 
library(ROAuth)
library(jsonlite)
library(httpuv)
suppressWarnings(expr)
 ```

 - Step 2: Load the Dataset we grab by using R
 ```{r}
 twtr <- read.csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/raw data/twitterR.csv")
 head(twtr,5)
 ```

 - Step 3: Find the dataset with NA missing value
 
 Since R collects lots of basic infor of users and tweet situation, we just need to check whether there are missing value in text variable. Through checking, there is no missing value in the text.
 ```{r}
 sum(is.na(twtr$text))
 ```
 
 - Step 4: Pre-processing the text info in the dataset
```{r}
twittertext = Corpus(VectorSource(twtr$text))
toSpace = content_transformer(
              function (x, pattern)
              gsub(pattern, " ", x))
twittertext1 = tm_map(twittertext, toSpace, "/")
twittertext1 = tm_map(twittertext, toSpace, "@")
twittertext1 = tm_map(twittertext, toSpace, "#")
twittertext1 = tm_map(twittertext1, content_transformer(tolower))
twittertext1 = tm_map(twittertext1, removeNumbers)
twittertext1 = tm_map(twittertext1, stripWhitespace)
removeURL <- function(x) gsub("http[[:alnum:]]*", "", x)
text = tm_map(twittertext, removeURL)
head(text)
```

 - Step 5: count the words freqency
```{r}
term = TermDocumentMatrix(text)
m = as.matrix(term)
v = sort(rowSums(m), 
         decreasing = TRUE)
d = data.frame(word = names(v),
               freq = v)
head(d, 10)
```

 - Step 6: saved as csv for python sentiment analysis. 
```{r}
write.csv(d,"/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/cleaned data/countwordR.csv")
```

 - Step 7: using python and the similar step as the previous data processing to do opinion mining

In [ ]:
twtr = pd.read_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/raw data/twitterR.csv")

# define the function
def getSentiments(df):
    sid = SentimentIntensityAnalyzer()
    tweet_str = ""
    tweetscore = []
    for tweet in df['text']:
        tweet_str = tweet_str + " " + tweet
        score = sid.polarity_scores(tweet_str)
        tweetscore.append(score)
    return tweetscore

# call the function above to see the result score and form a data frame to record
sentiment = getSentiments(twtr)
texts = pd.DataFrame(twtr.text)
stmscore = pd.DataFrame.from_dict(sentiment)
stmscore.head()

# relate the text and score for better view
txtscore = pd.concat([texts,stmscore],axis=1)
print(txtscore.head())

# export the csv for future analysis
txtscore.to_csv("/Users/apple/Desktop/anly-501-project-HuitingSong-1/501-project-website/data/cleaned data/rstmscore.csv")